In [1]:
%%capture
!pip install pyctcdecode
!python -m pip install pypi-kenlm
!pip install jiwer

![](https://developer-blogs.nvidia.com/wp-content/uploads/2019/12/automatic-speech-recognition_updated.png)

![](https://www.researchgate.net/profile/Diana-Militaru/publication/299594444/figure/fig1/AS:346834426974208@1459703179403/The-block-diagram-of-an-automatic-speech-recognition-and-understanding-system.png)

in this notebook we will try to demonstrate how to calculate CER,WER metric on validation dataset using xls-r wav2vec2 model,we will be using public best available pretrained model from huggingface to demonstrate the metric calculation process. for understanding how to train wav2vec2 on this dataset please check our past work [wav2vec2 starter](https://www.kaggle.com/code/nazmuddhohaansary/wave2vec2-starter-for-dl-sprint-commonvoice)

# Imports

In [2]:
import os
import numpy as np
from tqdm.auto import tqdm
from glob import glob
from transformers import AutoFeatureExtractor, pipeline
import pandas as pd
import librosa
import IPython
from datasets import load_metric
from tqdm.auto import tqdm
from torch.utils.data import Dataset, DataLoader
import torch
import gc
import wave
from scipy.io import wavfile
import scipy.signal as sps

tqdm.pandas()
import warnings
warnings.filterwarnings("ignore")



# Configs

In [3]:
#according to our experiment this is the best model -> arijitx/wav2vec2-xls-r-300m-bengali
class CFG:
    model_name = 'arijitx/wav2vec2-xls-r-300m-bengali' #arijitx/wav2vec2-large-xlsr-bengali,arijitx/wav2vec2-xls-r-300m-bengali, Tahsin-Mayeesha/wav2vec2-bn-300m
    valid_df_path = '../input/dlsprint/validation.csv'
    sample_sub_df_path = '../input/dlsprint/sample_submission.csv'
    valid = "../input/dlsprint/validation_files/"
    test = "../input/dlsprint/test_files/"
    valid_wav = '../input/validation-fileswav-format/validation_files_wav/'
    test_wav = '../input/test-wav-files-dl-sprint/test_files_wav/'
    batch_size = 48#not using this param now
    single_SPEECH_FILE = "../input/dlsprint/validation_files/common_voice_bn_30620258.mp3"
    



# single sample inference demo

In [4]:
asr = pipeline("automatic-speech-recognition", model=CFG.model_name, device=0)
feature_extractor = AutoFeatureExtractor.from_pretrained(
        CFG.model_name, cache_dir=None, use_auth_token=False
    )
speech, sr = librosa.load(CFG.single_SPEECH_FILE, sr=feature_extractor.sampling_rate)
prediction = asr(
            speech, chunk_length_s=112, stride_length_s=None
        )

pred = prediction["text"]
pred


Downloading:   0%|          | 0.00/1.99k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.18G [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/287 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.13k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/78.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/3.48G [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/78.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/57.2M [00:00<?, ?B/s]

'তার পিতার নাম কালীপ্রসন্ন ভট্টাচার্য'

# check the original audio

In [5]:
IPython.display.Audio(CFG.single_SPEECH_FILE)

# Fix paths

In [6]:
df = pd.read_csv('../input/dlsprint/validation.csv')
directory ="../input/dlsprint/validation_files/"
df["path"]=df["path"].progress_apply(lambda x:os.path.join(directory,str(x)))
df.head(3)

  0%|          | 0/7747 [00:00<?, ?it/s]

,client_id,path,sentence,up_votes,down_votes,age,gender,accents,locale
0,c0494c8220a53efec93f188e32be94d3c1832c48117423...,../input/dlsprint/validation_files/common_voic...,"কৃষি, সেবা, রেমিটেন্স, ব্যবসা ও অন্যান্য।",3.0,0.0,NaN,NaN,NaN,bn
1,c0494c8220a53efec93f188e32be94d3c1832c48117423...,../input/dlsprint/validation_files/common_voic...,তিনি ছিলেন চাকমা ভাষার প্রথম আধুনিক গীতিকার।,6.0,1.0,NaN,NaN,NaN,bn
2,c06b36547c86713d53bb2bf696a34b696de586c5ab1aa9...,../input/dlsprint/validation_files/common_voic...,ইংরেজির সাথে সাথে তাদের হিন্দী ও সংস্কৃত শিক্ষ...,3.0,1.0,NaN,NaN,NaN,bn


# Custom dataset class

librosa with mp3 is super slow,so we will be using wav files for faster inference

In [7]:
class bn_asr_Dataset(Dataset):
    '''
    args:
        df      : path of the dataframe
        dir     : directory of sound files
    '''
    def __init__(self,df,dir):
        self.df = pd.read_csv(df)
        self.dir = dir

    def __len__(self):
        return len(self.df)

    def __getitem__(self, i):
   
        #speech, _ = librosa.load(self.dir+self.df.path[i], sr=feature_extractor.sampling_rate) 
        path = self.dir+self.df.path[i]
        path = os.path.splitext(path)[0]+'.wav'
        # Read file
        sampling_rate, data = wavfile.read(path)
        # Resample data
        number_of_samples = round(len(data) * float(feature_extractor.sampling_rate) / sampling_rate)
        speech = sps.resample(data, number_of_samples)
        return speech
  


# making prediction on whole validation set

In [8]:
%%time
#single image inference
''' 
#super slow inference...

predictions = []
references = []
for i in range(len(df.path)):
    speech, sr = librosa.load(df.path[i], sr=feature_extractor.sampling_rate)
    prediction = asr(speech, chunk_length_s=112, stride_length_s=None)
    pred = prediction["text"]
    predictions.append(pred)
    references.append(df.sentence[i])
    
print(len(predictions),len(references))
'''

df = pd.read_csv(CFG.valid_df_path)
valid_dataset = bn_asr_Dataset(CFG.valid_df_path,CFG.valid_wav)#CFG.valid
predictions = []
references = []
# for i,pred_sentence in enumerate(tqdm(asr(valid_dataset, chunk_length_s=112, stride_length_s=None,batch_size=CFG.batch_size), total=len(valid_dataset))):
#     references.append(df.sentence[i])
#     predictions.append(pred_sentence['text'])
    
for i in range(len(valid_dataset)):
    pred = asr(valid_dataset.__getitem__(i), chunk_length_s=112, stride_length_s=None)
    references.append(df.sentence[i])
    predictions.append(pred['text'])
  

CPU times: user 19min 28s, sys: 4.97 s, total: 19min 33s
Wall time: 21min 29s


In [9]:
!nvidia-smi

Sun Jul  3 23:32:17 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.82.01    Driver Version: 470.82.01    CUDA Version: 11.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   54C    P0    42W / 250W |   3419MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [10]:
torch.cuda.empty_cache()
gc.collect()
!nvidia-smi

Sun Jul  3 23:32:27 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.82.01    Driver Version: 470.82.01    CUDA Version: 11.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   52C    P0    35W / 250W |   2167MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

# calculating metric on whole validation set

In [11]:

df = pd.DataFrame(columns=['predictions', 'references'])
df.predictions = predictions
df.references = references
df.to_csv('./results.csv',index = False) #use it for error analysis and other stuffs
df.head(10)

,predictions,references
0,কৃষি সেবা রেমিটেন্স ব্যবসা ও অন্যান্য,"কৃষি, সেবা, রেমিটেন্স, ব্যবসা ও অন্যান্য।"
1,তিনি ছিলেন চাকমা বাসার প্রথ মাধ্যমের গীতিকা।,তিনি ছিলেন চাকমা ভাষার প্রথম আধুনিক গীতিকার।
2,ইংরেজি সাথে সাথে তাদের হিন্দি ও সংস্কৃত শিক্ষা...,ইংরেজির সাথে সাথে তাদের হিন্দী ও সংস্কৃত শিক্ষ...
3,শিক্ষার ধারণ তার প্রথম আন্তর্জাতিক উইক েটশিকার...,শিখর ধাওয়ান তার প্রথম আন্তর্জাতিক উইকেট শিকার...
4,চতুর্থ সপ্তাহ থেকে অবস্থার উন্নতি হতে থাকে কিন...,"চতুর্থ সপ্তাহ থেকে অবস্থার উন্নতি হতে থাকে,কিন..."
5,এখানে তিনি এয়াকিনসের অধীনে অধ্যয়ন করেছেন,এখানে তিনি এয়াকিনসের অধীনে অধ্যয়ন করেছিলেন।
6,শীতকালীন গেমস এ কোন কোন পদক জিততে পারেনি।,শীতকালীন গেমসে এখনো কোন পদক জিততে পারেনি।
7,অবশেষে তিনি কাবুলে ফিরে আসেন,"অবশেষে, তিনি কাবুলে ফিরে আসেন।"
8,তিনি অ্যাথেলসেএসে চক্রটির শীর্ষ হন,তিনি এথেন্সে এসে সক্রেটিসের শিষ্য হন।
9,তিন বছর বয়সে তার বাবা মারা যান,তিন বছর বয়সে তাঁর বাবা মারা যান।


In [12]:
cer = load_metric("cer")
wer = load_metric("wer")

cer_score = cer.compute(predictions=predictions, references=references)
print("validation cer_score -> ",cer_score)
wer_score = wer.compute(predictions=predictions, references=references)
print("validation wer_score -> ",wer_score)

validation cer_score ->  0.09787704766628824
validation wer_score ->  0.30921300101701055


# Submission

In [13]:
df = pd.read_csv('../input/dlsprint/sample_submission.csv')
len(df.path)

7747

In [14]:
%%time

test_dataset = bn_asr_Dataset(CFG.sample_sub_df_path,CFG.test_wav)

# for i,prediction in enumerate(tqdm(asr(test_dataset, chunk_length_s=112, stride_length_s=None,batch_size=CFG.batch_size), total=len(test_dataset))):
#     df.sentence[i] = prediction["text"]
    
for i in range(len(test_dataset)):
    pred = asr(test_dataset.__getitem__(i), chunk_length_s=112, stride_length_s=None)
    df.sentence[i] = pred["text"]
      

CPU times: user 20min 38s, sys: 5.74 s, total: 20min 44s
Wall time: 23min 41s


In [15]:
df.head(3)

,path,sentence
0,common_voice_bn_31675220.mp3,এছাড়াও নিউজল্যান্ড এই ক্রিকেট দলের হয়ে খেলছে...
1,common_voice_bn_31513116.mp3,এই ফল পাখি রাখায় কিন্তু নিজে পড়ে থাকা ফল খেল...
2,common_voice_bn_31558126.mp3,জন পরিকল্পিত


In [16]:
df.to_csv('./submission.csv',index = False)
df.sentence[1]

'এই ফল পাখি রাখায় কিন্তু নিজে পড়ে থাকা ফল খেলে কুকুর অসুস্থ হয়ে পড়ে।'

In [17]:
IPython.display.Audio('../input/dlsprint/test_files/common_voice_bn_31675220.mp3')

In [18]:
df.sentence[0]

'এছাড়াও নিউজল্যান্ড এই ক্রিকেট দলের হয়ে খেলছেন তিনি।'

![](https://images.unsplash.com/photo-1499744937866-d7e566a20a61?ixlib=rb-1.2.1&ixid=MnwxMjA3fDB8MHxwaG90by1wYWdlfHx8fGVufDB8fHx8&auto=format&fit=crop&w=870&q=80)